# Assignment 3: Improving the Index

In this assignment, we will improve the search index and query functions from the previous assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10, sqrt

Summaries_file = 'fungus_Summaries.pkl.bz2'
Abstracts_file = 'fungus_Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [4]:
inverted_index = defaultdict(list)

for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. We won't go much into the details of tokenization and linguistic analysis here, because we also want to focus on scoring and ranking below. Therefore, we are using an existing library for tokenizatoin and stemming, namely the NLTK package. The following line will install NLTK if necessary (or you have to follow [these instructions](http://www.nltk.org/install.html) if that doesn't work):

In [5]:
! pip install --user nltk

In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
stemmer = EnglishStemmer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print('INPUT TEXT:\n ', s)

print('TOKENIZE: ', tokenize(s))
print('WORD TOKENIZE: ', word_tokenize(s))

INPUT TEXT:
  Good muffins cost $3.88
in New York.  Please buy me two of them.

Thanks.
TOKENIZE:  ['Good', 'muffins', 'cost', '$3.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
WORD TOKENIZE:  ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juann\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
print(stemmer.stem("processes"))

process


## Ranking

Another important method to improve our search results is to rank them, which can be done by calculating a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, which comes with several variants, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- num_documents: The number of documents in our index

In [8]:
tf_matrix = defaultdict(Counter)

for doc_id in Summaries.keys():
    tokens = preprocess(tokenize(Summaries[doc_id].title))
    if (doc_id in Abstracts):
        tokens.extend(preprocess(tokenize(Abstracts[doc_id])))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Summaries))

Let's test these functions with some examples:

In [9]:
print(tf('amsterdam', 22679594))
print(df('amsterdam'))
print(num_documents)

3.0
5.0
54751.0


With these helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

# Tasks

**Your name:** Juan Rodriguez Peinado 

### Task 1

Implement in the code block below the `smarter_tokenize_and_preprocess` function using NLTK's functions for tokenization and stemming. 

In [10]:
# Smarter linguistic processing

# Your code here:

def smarter_tokenize_and_preprocess(text):
#  ...
    query = word_tokenize(text)
    singles = [stemmer.stem(element) for element in query]
    return singles
# To test it:
print(smarter_tokenize_and_preprocess("He buys many books, some about I.R., for less than $1.50!"))

['he', 'buy', 'mani', 'book', ',', 'some', 'about', 'i.r.', ',', 'for', 'less', 'than', '$', '1.50', '!']


Now we can make a smarter index based on this function. For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time. (You don't need to change or add anything in the code block below. Just leave it as it is.)

In [11]:
# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(list)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = list(key for key in Summaries.keys() if 28000000 <= key < 29000000)

# Building our smarter index:
for id in sorted(subset_of_ids):
    term_set = set(smarter_tokenize_and_preprocess(Summaries[id].title))
    if id in Abstracts:
        term_set.update(smarter_tokenize_and_preprocess(Abstracts[id]))
    for term in term_set:
        smarter_index[term].append(id)
#print(smarter_index)

Now implement the `smarter_and_query` function, based on the two functions `smarter_tokenize` and `smarter_preprocess` you defined above and accessing our new index `smarter_index`. You can start from copying the code for `and_query` from the last assignment. For that to work, you'll also have to copy the code for the `and_merge` function from the last assignment.

In [12]:
# Smarter and_query based on the smarter tokenize and preprocess functions

# Your code here:

def and_merge(sorted_list1, sorted_list2):
    merged_list = []
    
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    lst = list1+list2

    # ---
    
    dupes = [x for n, x in enumerate(lst) if x in lst[:n]]
    # ---
    return dupes


def and_query(a_query):
    merge_list = []
    elem = smarter_tokenize_and_preprocess(a_query)
    merge_list = smarter_index[elem[0]]
    print(elem)
    for word in elem[1:]:
        merge_list = and_merge(merge_list, smarter_index[word])
    
    # ---
    return merge_list
        

def smarter_and_query(query_string):
#  ...
    a_query = and_query(query_string)
    # --- 
    return a_query
   
    

query = 'basidiomycete Heterobasidion annosum'
searching_query = smarter_and_query(query)
print(searching_query)
for elemen in searching_query:
        display_summary(elemen, show_abstract = True)


['basidiomycet', 'heterobasidion', 'annosum']
[28801666]


### Task 2

Run the query "infection range expansions" with the new `smarter_and_query` function from task 1. Does it return paper *28425998*? Explain what our new smarter function specifically contributes to the result (as compared to our previous naive implementations for tokenization and preprocessing)?

In [13]:
# Smarter and_query based on the smarter tokenize and preprocess functions

# Your code here:

def and_merge(sorted_list1, sorted_list2):
    merged_list = []
    
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    lst = list1+list2

    # ---
    
    dupes = [x for n, x in enumerate(lst) if x in lst[:n]]
    # ---
    return dupes


def and_query(a_query):
    merge_list = []
    elem = smarter_tokenize_and_preprocess(a_query)
    merge_list = smarter_index[elem[0]]
    for word in elem[1:]:
        merge_list = and_merge(merge_list, smarter_index[word])
    
    # ---
    return merge_list
        

def smarter_and_query(query_string):
#  ...
    a_query = and_query(query_string)
    # --- 
    return a_query
   
    

query = 'infection range expansions'
searching_query = smarter_and_query(query)
#print(searching_query)
for elemen in searching_query:
        display_summary(elemen, show_abstract = True)

**Answer:** Well, basically in the tokenize and preprocessing functions, it does not alterate query at all, the tokenize function just split the 'text' but it is already splitted and the preprocessing function just lower case the strings that are given in the query. It barely changes the input.

### Task 3

Now we move to a different subject and use our old index again. That is, we **don't** use the smarter functions defined above for tasks 3 to 5!

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using `tf(t,d)`, `df(t)` and `num_documents` as defined above. To do this, first implement a function `idf(t)` to calculate the inverse document frequency, and then use this function to calculate the full tf-idf. Use the _add-one-smoothing_ version of idf, so we don't run into problems with terms that don't appear in the collection at all. The relevant formulas can be found on the lecture slides. Use tf-idf with plain (non-logarithmic) term frequency, as applied by scoring variant `ntn`. Test your function with the examples shown below. You can use the `log10(n)` function to calculate the base 10 logarithm.

Again, use our old (non-smart) index for this task and the tasks below, and **not** the functions defined in tasks 1 and 2.

In [10]:
# Your code here:

def idf(t):
    result = log10(num_documents/ df(t)) 
    #-
    return result
    
def tfidf(t,d):
    result_idf = idf(t)
    result_tf = tf(t,d)
    result_tf_idf = result_idf * result_tf
    #-
    return format(result_tf_idf, ".2f") #this might not be necessary, ask TA about it

print(tfidf('europe', 28425998))
print(tfidf('chytrid', 28425998))
print(tfidf('this', 28425998))

2.29
4.35
1.31


### Task 4

Create a function `query_ntn_nnn(query_string)`, which accepts as input a single query string of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying variant `ntn.nnn`, as above (see the formula for the `ntn.nnn` version of scoring on the lecture slides). Use an auxiliary function `score_ntn_nnn` to calculate the score. The results should be shown in descending order by score.

You can start by copying your functions `or_merge` and `or_query` from assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created above.

Demonstrate your function by giving it the exemplary query string "evolution of resistance in the context of rice".

In [15]:
def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Summaries))

def idf(t):
    return (log10(num_documents/ (df(t))))
    
def tfidf(t,d):
    #-
    return (idf(t) * tf(t,d))


def or_merge(sorted_list1, sorted_list2):
    
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    #-
    i = 0
    j = 0
    result = []
    #-
    while (i < len(list1) and j < len(list2)):
        if list1[i] == list2[j]:
            result.append(list1[i])
            i +=1
            j +=1
        elif list1[i] <= list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1

    remained = list1[i:] + list2[j:]
    result.extend(remained)
    #-
    return (result)

                   
def or_query(a_query):
    merge_list = []
    merge_list = inverted_index[a_query[0]]
    #-
    for word in a_query[1:]:
        merge_list = or_merge(merge_list, inverted_index[word])
    #-
    return (merge_list)

def score_ntn_nnn(query_words, doc_id):
    result = 0
    #-
    for i in query_words:
        score = tfidf(i,doc_id)
        result += score
    return (result)

def query_ntn_nnn(query_string):
    elem = preprocess(tokenize(query_string))
    docs_id = or_query(elem)
    final_result = []
    #-
    for doc_id in docs_id:
        result = score_ntn_nnn(elem, doc_id)
        final_result .append((result, doc_id))
    return (sorted(final_result , reverse = True)[:10])


query = 'evolution of resistance in the context of rice'
search_query = query_ntn_nnn(query)  

print(search_query)

for (rank,doc_id) in search_query:
        display_summary(doc_id, show_abstract = True)
        print('Total score is {}'.format(rank))

[(36.053067402310035, 25324853), (26.126036524122867, 26720072), (24.68529182488877, 23888278), (24.59229117200141, 30617050), (24.37124802448904, 26635817), (24.27948205909175, 26158382), (24.248082181228728, 2907262), (22.410821723511884, 17022177), (21.69203744052453, 2016048), (21.45153796294144, 25547964)]


Total score is 36.053067402310035


Total score is 26.126036524122867


Total score is 24.68529182488877


Total score is 24.59229117200141


...

Total score is 24.27948205909175


Total score is 24.248082181228728


Total score is 22.410821723511884


Total score is 21.69203744052453


Total score is 21.45153796294144


### Task 5

In this last task, you should create a second version of the query function from Task 4, called `query_ntc_ntc`. This second version should use, as its name suggests, variant `ntc.ntc` instead of `ntn.nnn`, and therefore apply the cosine similarity measure, in addition to applying _tf-idf_. For this, consult the formula for variant `nnc.nnc` on the lecture slides and adopt it to include the _idf_ metric (that is, add the `t` element of `ntc`). (You can drop the square root of |q| in the formula, as indicated on the slides.)

As a first step, we can calculate beforehand the length of all document vectors (because they don't depend on the query) for document vectors consisting of _tf-idf_ values. The code below does just that, assuming that you defined the function `tfidf(t,d)` above (don't change this code block, just run it):

In [16]:
tfidf_length_values = defaultdict(int)

for doc_id in Summaries.keys():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

We can now get the length of a document vector by calling `tfidf_length(d)`.

Based on this, you can now implement `query_ntc_ntc` in the code block below. You should again first define an auxiliary function, called `score_ntc_ntc`. You can start by copy-pasting the code from Task 4.

To output the results, use the provided `display_summary` function to make the output a bit more like the results page of a search engine. Lastly, demonstrate your `query_ntc_ntc` function with the same example query as above: "evolution of resistance in the context of rice"

In [17]:
def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Summaries))

def idf(t):
    return (log10(num_documents/ (df(t))))
    
def tfidf(t,d):
    #-
    return (idf(t) * tf(t,d))


def or_merge(sorted_list1, sorted_list2):
    
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    #-
    i = 0
    j = 0
    result = []
    #-
    while (i < len(list1) and j < len(list2)):
        if list1[i] == list2[j]:
            result.append(list1[i])
            i +=1
            j +=1
        elif list1[i] <= list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1

    remained = list1[i:] + list2[j:]
    result.extend(remained)
    #-
    return (result)

                   
def or_query(a_query):
    merge_list = []
    merge_list = inverted_index[a_query[0]]
    #-
    for word in a_query[1:]:
        merge_list = or_merge(merge_list, inverted_index[word])
    #-
    return (merge_list)

def score_ntn_nnn(query_words, doc_id):
    result = 0
    #-
    for i in query_words:
        score = tfidf(i,doc_id) * idf(i)
        result += score
    
    
    return (result / tfidf_length(doc_id))

def query_ntn_nnn(query_string):
    elem = preprocess(tokenize(query_string))
    docs_id = or_query(elem)
    final_result = []
    #-
    for doc_id in docs_id:
        result = score_ntn_nnn(elem, doc_id)
        final_result.append((result, doc_id))
    return (sorted(final_result, reverse = True)[:10])


query = 'evolution of resistance in the context of rice'
search_query = query_ntn_nnn(query)  

print(search_query)

for (rank,doc_id) in search_query:
    display_summary(doc_id, show_abstract = True)
    print('Total score is {}'.format(rank))
        

[(1.0459475433838634, 25324853), (1.024553773628267, 31309315), (0.9554419160053966, 23544090), (0.9088530689359053, 26635817), (0.8875285904197701, 23741492), (0.867611189057035, 27576685), (0.8593354479128119, 29679239), (0.829774618277997, 26158382), (0.7775706620791388, 30112588), (0.7556450694787762, 28154240)]


Total score is 1.0459475433838634


Total score is 1.024553773628267


Total score is 0.9554419160053966


Total score is 0.9088530689359053


...

Total score is 0.867611189057035


Total score is 0.8593354479128119


Total score is 0.829774618277997


Total score is 0.7775706620791388


Total score is 0.7556450694787762


# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.